# Capstone Project Notebook


Notebook for the Data Science Specialization Capstone 

In [1]:
#importing numpy and Pandas
import pandas as pd
import numpy as np

## Getting and parsing the Wikipedia page of postal codes of Canada
First, we import BeautifulSoup and urlopen to read and then parse the html page, we then filter out the tabe of postal codes using BeautifulSoup's find function.

Secondly, we convert the html table into a pandas dataframe.

In [2]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [3]:
#Read the Wekipedia html page of postal codes of Canada.
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = urlopen(url)
html = response.read()

In [4]:
#Filtering the table of postal codes of Canada
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table')
df = pd.read_html(str(table))[0]

In [5]:
#convert all the Not assigned entries of the data frame to NAN
df.replace("Not assigned", np.nan, inplace=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,NaN
9,M8A,NaN,NaN


In [6]:
#Remove all Boroughs with Not assigned entries
df.dropna(subset=["Borough"], inplace=True)
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,NaN
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [7]:
#Replace  all Neighbourhood with Not assigned entries with Boroughs Name.

df.Neighbourhood.fillna(df.Borough,inplace=True)
df.head(10)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [8]:
#join Neighbourhoods with the same Postcode to the same row.
g = df.groupby(['Postcode','Borough'])
sm = lambda lst: ', '.join(lst)
df = g.aggregate(sm)
df.reset_index(inplace=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [9]:
df.shape

(103, 3)

## Reading the Geolocation of the Neighbourhoods of Canada
We import geocoder to get the Lat and Long of each Neigbourhood.
However, the geocoder is not  working reliably so we use the provided CSV file.

In [10]:
import geocoder # import geocoder

In [11]:
df_latlng = pd.read_csv("Geospatial_Coordinates.csv")
df_latlng.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [12]:
df_latlng.shape

(103, 3)

In [13]:
#Using Inner product we join both dataframes(Neighbourhoods and Geoocations) on the Postal Codes columns.
df = pd.merge(df, df_latlng, left_on="Postcode", right_on="Postal Code", how="inner")
df.drop("Postal Code", axis=1, inplace=True)
df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Visualizing and Clustering the Neigbourhoods
We use folium for map Visualization and Kmeans to cluster the neighbourhoods

In [14]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

In [15]:
#Get the lat and lng of Toronto Canada
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [16]:
#Show Toronto, Canda on the map.
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [17]:
#cluster the neighbourhoods into 6 clusters using the lat and lng coordinates.
kclusters = 6

df_clst = df[["Latitude","Longitude"]]

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_clst)

In [18]:
df["clusters"] = kmeans.labels_
df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,clusters
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,2
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,2
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,5
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,2
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,5


In [21]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10.5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'], df['Longitude'], df['Neighbourhood'], df['clusters']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Conclusion
As expected the Clusters of the Neighbourhoods are based on their proximity to one another as we have used the lat and lng coordinate for the clustering.